# Data Preprocessing : Creation of the Daily and the Hourly Datasets

In [ ]:
#libraries we need
import datetime
import pandas as pd
import numpy as np
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm


In [ ]:
# import base dataset
data = pd.read_csv('data/household_data_1min_singleindex.csv')

#### industrial_components.csv

In [ ]:
industrial = data.filter(like='industrial3')
industrial['time'] = data.filter(like='cet_cest_timestamp')
industrial = industrial.drop(columns = ['DE_KN_industrial3_compressor', 'DE_KN_industrial3_ev', 'DE_KN_industrial3_grid_import', 'DE_KN_industrial3_ventilation', 'DE_KN_industrial3_pv_facade', 'DE_KN_industrial3_pv_roof'])
industrial.replace(0, np.nan, inplace=True)
industrial = industrial.dropna()

In [ ]:
industrial_clean = pd.DataFrame()

industrial_clean['rooms'] = industrial.filter(like='area').sum(axis=1)
industrial_clean['machine'] = industrial.filter(like='machine').sum(axis=1)
industrial_clean['cooling'] = industrial.filter(like='cooling').sum(axis=1)
industrial_clean['appliances'] = industrial[['DE_KN_industrial3_refrigerator', 'DE_KN_industrial3_dishwasher']].sum(axis = 1)
industrial_clean['time'] = industrial['time']

In [ ]:
industrial_clean.to_csv('data/industrial_components.csv', index = False)

#### industrial_total.csv

In [ ]:
industrial = data.filter(like='industrial3')
industrial = industrial.drop(columns = ['DE_KN_industrial3_ev', 'DE_KN_industrial3_grid_import', 'DE_KN_industrial3_pv_facade', 'DE_KN_industrial3_pv_roof'])
industrial['time'] = data.filter(like = 'cet_cest_timestamp')
industrial.replace(0, np.nan, inplace=True)
industrial = industrial.dropna()

In [ ]:
consumption = pd.DataFrame()
consumption['cumulative'] = industrial.drop(columns = 'time').sum(axis = 1)
consumption['difference'] = consumption['cumulative'].diff()
consumption['time'] = industrial['time']

In [ ]:
consumption.to_csv('data/industrial_total.csv', index = False)

#### daily_data.csv

In [ ]:
#dataset we need : industrial_components
df = pd.read_csv('data/industrial_components.csv')

In [ ]:
#we split the date into 4 columns
time1 = df['time'].str.split('-', expand=True)
time1.columns = ['Year', 'Month', 'Time']
time2 = time1['Time'].str.split('T', expand=True)
time2.columns = ['Day', 'Time']
time2['Time'] = time2['Time'].str.replace(r'\D+0100', '')
del time1['Time']
time_merged = pd.merge(time1, time2, left_index=True, right_index=True)

In [ ]:
#we create a dataset called df2 that will add columns about the date on the df1 dataset
df2 = pd.merge(df, time_merged, left_index=True, right_index=True)
df2['Time'] = df2['Time'].str.replace(r'\D+0200', '')
df2['time'] = df2['time'].str.replace(r'\D+0200', '')
df2['time'] = df2['time'].str.replace(r'\D+0100', '')

In [ ]:
#we do the same with the column time to have a column with the date
tip = df2['time'].str.split('T', expand=True)
tip.columns = ['Date', 'Hour']
del tip['Hour']

In [ ]:
#we add a column with the day name of the date
df2 = pd.merge(df2, tip, left_index=True, right_index=True)
df2['date'] = pd.to_datetime(df2['Date'], errors='coerce')
del df2['Date']
df2['Day_of_week'] = df2['date'].dt.day_name()

In [ ]:
#we create a dataset called df3 that will take only the value of the consumption
df3 = df2.drop(['time','Year', 'Month', 'Day', 'Time', 'date'], axis = 1)

df3['rooms_diff'] = df3['rooms'].diff()
df3['machine_diff'] = df3['machine'].diff()
df3['cooling_diff'] = df3['cooling'].diff()
df3['appliances_diff'] = df3['appliances'].diff()

df3['rooms_diff'] = df3['rooms_diff'].fillna(0)
df3['machine_diff'] = df3['machine_diff'].fillna(0)
df3['cooling_diff'] = df3['cooling_diff'].fillna(0)
df3['appliances_diff'] = df3['appliances_diff'].fillna(0)

In [ ]:
#we create the dataset with the consumption for each day (1440 minutes per day)

parameters = {'rooms_diff': 'sum', 
'machine_diff': 'sum', 
'cooling_diff': 'sum', 
'appliances_diff': 'sum'}

daily_data = df3.groupby(df3.index // 1440).agg(parameters)

daily_data['rooms_diff_mean'] = daily_data['rooms_diff']/1440
daily_data['machine_diff_mean'] = daily_data['machine_diff']/1440
daily_data['cooling_diff_mean'] = daily_data['cooling_diff']/1440
daily_data['appliances_diff_mean'] = daily_data['appliances_diff']/1440

new_date = df2['date'].iloc[1439:]
daily_data['date'] = new_date.unique()
daily_data['Day_of_week'] = daily_data['date'].dt.day_name()

In [ ]:
daily_data.to_csv('data/daily_data.csv')

#### hourly_data.csv

In [ ]:
##we create the dataset with the consumption for each day (60 minutes per hour)

parameter2 = {'rooms_diff': 'sum', 
'machine_diff': 'sum', 
'cooling_diff': 'sum', 
'appliances_diff': 'sum'}
hourly_data = df3.groupby(df3.index // 60).agg(parameter2)

hourly_data['rooms_diff_mean'] = hourly_data['rooms_diff']/60
hourly_data['machine_diff_mean'] = hourly_data['machine_diff']/60
hourly_data['cooling_diff_mean'] = hourly_data['cooling_diff']/60
hourly_data['appliances_diff_mean'] = hourly_data['appliances_diff']/60

In [ ]:
hourly_data.to_csv('data/hourly_data.csv')